In [ ]:
from ipaddress import IPv4Address
import csv
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
train_df = csvProcessing('./train.csv')
mal_df = train_df[train_df['Label']==1]
ben_df = train_df[train_df['Label']==0]
mal_target_ip = mal_df['target_ip']
ben_target_ip = ben_df['target_ip']


In [ ]:
target_mal_ip = mal_df['target_ip']
tmp = []
for ip in target_mal_ip:
        for attack in grey[ip][1]:
            tmp.append(attack)
tmp1 = set(tmp)
attack_freq = {}
for attack in tmp1:
    attack_freq[attack] = 0
for attack in tmp:
    attack_freq[attack] +=1


sorted_attack_freq = dict(sorted(attack_freq.items(), key=lambda x: x[1]))
a = list(sorted_attack_freq.keys())
for attack in a:
    if sorted_attack_freq[attack]<50:
        del(sorted_attack_freq[attack])
attacks = list(sorted_attack_freq.keys())

target_mal_ip = mal_df['target_ip']
attack_labels = []
for ip in target_mal_ip:
    for attack in attacks:
        if attack in grey[ip][1]:
            attack_labels.append(attack) 
            break
mal_df['attack_label'] = attack_labels       

In [ ]:
attack_label_set = set(mal_df['attack_label'])

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors




In [ ]:
from sklearn.manifold import TSNE

i=0
column_names = mal_df.columns.values.tolist()
denoised_mal_df = pd.DataFrame(columns=column_names)
for attack in attack_label_set:
    df_sample = mal_df[mal_df['attack_label'] == attack]
    if len(df_sample) > 75000:
        df_sample = df_sample.sample(n=75000,random_state=1)

    target_ip_list = df_sample['target_ip']
    start_time = df_sample['start_time']
    end_time = df_sample['end_time']
    labels = df_sample['Label']
    attack_label = df_sample['attack_label']
    
    df_sample = df_sample.drop(labels=['target_ip','start_time','end_time','Label','attack_label'],axis=1)

    # 데이터 스케일링
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df_sample)
    dbscan = DBSCAN(eps=0.02, n_jobs=10,min_samples=20,metric='cosine')
    dbscan_labels = dbscan.fit_predict(scaled_data)

    # 클러스터링 결과를 데이터프레임에 추가
    df_sample['dbscan_label'] = dbscan_labels
    df_sample['target_ip'] = target_ip_list
    df_sample['start_time'] = start_time
    df_sample['end_time'] = end_time
    df_sample['Label'] = labels
    df_sample['attack_label'] = attack_label
    tmp_df = df_sample[df_sample['dbscan_label']!=-1]
    db_labels_set = set(tmp_df['dbscan_label'])
    df_sample = None
    for db in db_labels_set:
        a_df = tmp_df[tmp_df['dbscan_label']==db]

        if len(a_df)>500 :
            denoised_mal_df = pd.concat([denoised_mal_df,a_df])
        a_df = None
    tmp_df = None

In [ ]:
denoised_mal_df.to_csv('./denoise_train_attack1.csv',index=False)